In [1]:
import os
os.getcwd()

'/home/davis/isaac/apps/samples/python_api'

In [2]:
os.chdir("../../..")
os.getcwd()

'/home/davis/isaac'

In [3]:
CASK_PATH='</path/to/cask/>'
CHANNEL='color'

from engine.pyalice import Application, Codelet
import io, ipywidgets, IPython
from PIL import Image
# View the images
image_widget = ipywidgets.Image(height=720, width=1280)

ImportError: cannot import name 'PybindMessage'

In [ ]:
class Viewer(Codelet):
    def __init__(self):
        self.widget=None
    
    def start(self):
        self.rx = self.isaac_proto_rx("ColorCameraProto", "color")
        self.tick_on_message(self.rx)

    def tick(self):
        msg = self.rx.message
        if self.widget is None:
            return
        t = msg.tensor
        bts = io.BytesIO()
        Image.fromarray(t).save(bts, 'png')
        self.widget.value = bts.getvalue()

In [ ]:
IPython.display.display(image_widget)

app = Application()
 
replay_node = app.add('replay')
replay_component = replay_node.add(app.registry.isaac.alice.Replay)
replay_component.config['cask_directory'] = CASK_PATH
replay_component.config['report_success'] = True

viewer_node = app.add('viewer')
viewer_component = viewer_node.add(Viewer)

app.connect(replay_component, 'color', viewer_component, 'color')

app._pycodelet_frontends[viewer_component].widget=image_widget

app.run(20.0)

In [ ]:
app = Application()

app.load_module('viewers')

replay_node = app.add('replay')
replay_component = replay_node.add(app.registry.isaac.alice.Replay)
replay_component.config['cask_directory'] = CASK_PATH
replay_component.config['report_success'] = True

viewer_node = app.add('viewer')
viewer_component = viewer_node.add(app.registry.isaac.viewers.ColorCameraViewer)
widget = viewer_node.add(app.registry.isaac.sight.SightWidget)
widget.config['type'] = '2d'
widget.config['channels'] = [{'name':'viewer/ColorCameraViewer/Color', 'active': True}]
widget.config['dimensions'] = [int(720), int(1280)]

app.connect(replay_component, 'color', viewer_component, 'color_listener')

app.run(30.0)